In [1]:
# --LIBRARY IMPORTS--

import os 

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils.parametrize as parametrize

from model_architecture import GPTModel
import LoRA_parameterizer as lp

# initializing dailymail dataset and device

device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_df = pd.read_csv(os.path.join('../dataset/', 'dailymail/train.csv'))
val_df = pd.read_csv(os.path.join('../dataset/', 'dailymail/validation.csv'))
test_df = pd.read_csv(os.path.join('../dataset/', 'dailymail/test.csv'))

print(device)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

#initializing and loading model

model = GPTModel()

model_state = torch.load(os.path.join('../models/subset19/', 'mp_rank_00_model_states.pt'), map_location = device)

missing, unexpected = model.load_state_dict(model_state['module'], strict = False)

print("Missing keys:", missing)
print("Unexpected keys:", unexpected)

model = model.to(device)

cpu
(287113, 3)
(13368, 3)
(11490, 3)
Missing keys: []
Unexpected keys: []


In [2]:
# defining dataset class to build dataset from dataframes

class BuildDataSet(Dataset):

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        # get a single item
        item = self.df.iloc[idx]

        # return its text and highlight
        return {
            'text': item['text'],
            'highlights': item['highlights']
        }

In [3]:
# creating  train, test, val dataloaders for dailymail

train_dl = DataLoader(BuildDataSet(train_df), batch_size = 32, shuffle = True)

val_dl = DataLoader(BuildDataSet(val_df), batch_size = 32, shuffle = True)

test_dl = DataLoader(BuildDataSet(test_df), batch_size = 16, shuffle = True)

In [4]:
optimizer = model.optimizer

tokenizer = model.config.tokenizer

eos_token = model.eos_token_idx

pad_token = model.pad_token_idx

sos_token = tokenizer.convert_tokens_to_ids('<|startoftext|>')

# using unk token as context becuase I am an idiot and forgot to add a context token in the tokenizer

context_token = tokenizer.convert_tokens_to_ids('<|unknown|>') 

print("e, p, s, c : ", eos_token, pad_token, sos_token, context_token)

e, p, s, c :  50256 50258 50257 50259


In [ ]:
# copy the original weights and biases into a dictionary to compare post SFT and make sure they don't get lost along the way

original_weights = {}

for name, param in model.named_modules():

    if(hasattr(param, "weight")):

        original_weights[name] = param.weight.clone().detach()
        print("copied layer: ", name,"of size: ", param.weight.shape)

        print(original_weights[name])

In [ ]:
# parametrizing the model with LoRA

model = lp.apply_lora_to_model(model, device, rank = 1, alpha = 1)

lora_elements=0

for name, param in model.named_modules():

    if parametrize.is_parametrized(param, "weight"):
        lora_elements += param.parametrizations.weight[0].lora_A.nelement() +param.parametrizations.weight[0].lora_B.nelement()
print(f"number of lora parameters = {lora_elements}")

In [ ]:
# freezing the non-LoRA layers

for name, param in model.named_parameters():

    # freezing all non-LoRA and LoRA layers and matrices

    param.requires_grad = False

    print(f"Frozen layer {name}")

# check if LoRA is not in name

for name, param in model.named_modules():

    if hasattr(param, 'parametrizations'): #and hasattr(param.parametrizations, "weight"):

        print(f"un-freezing layer {name}")

        LoRA_weights = param.parametrizations["weight"][0]

        if hasattr(LoRA_weights, 'lora_A') and hasattr(LoRA_weights, "lora_B"):

            print(f"unfreezing LoRA layer of {param}")

            LoRA_weights.lora_A.requires_grad = True
            LoRA_weights.lora_B.requires_grad = True                    


In [ ]:
# checking if the parameters are correctly frozen

trainable = []

frozen = []

for name, param in model.named_parameters():

    if param.requires_grad:
        trainable.append(name)
        print(f"trainable parameters include {name}")

    else:

        frozen.append(name)

for layer in frozen:

    print(f"frozen includes {layer}")